In [1]:
import tensorflow as tf
from elmo import elmo_embedding
import numpy as np
import os
from tqdm import tqdm
import json
from model_v2 import LanguageModel
import unicodedata
import re

In [2]:
with open('pos2idx.json', 'r') as inp:
    pos2idx = json.load(inp)
idx2pos = {v: k for k, v in pos2idx.items()}
with open('baomoi_punc/word2idx.json', 'r') as inp:
    word2idx = json.load(inp)
with open('baomoi_punc/char2idx.json', 'r') as inp:
    char2idx = json.load(inp)

In [3]:
def tagger(inputs, labels, seq_lens, n_units, n_classes, drop_i, drop_o, name='tagger', reuse=False, is_training=True):
    with tf.variable_scope(name, reuse=reuse):
        def __cell():
            cell = tf.contrib.rnn.GRUBlockCellV2(n_units, name='cell', reuse=reuse)
            if is_training:
                cell = tf.nn.rnn_cell.DropoutWrapper(cell=cell, input_keep_prob=1.0-drop_i, output_keep_prob=1.0-drop_o, variational_recurrent=True, input_size=inputs.shape[-1], dtype=tf.float32)
            return cell
        outputs, state = tf.nn.bidirectional_dynamic_rnn(__cell(), __cell(), inputs, seq_lens, time_major=False, dtype=tf.float32)
        outputs = tf.concat(outputs, axis=-1)
        s = tf.shape(outputs)
        W = tf.get_variable(name='W', shape=(2 * n_units, n_classes), initializer=tf.glorot_uniform_initializer(), trainable=True)
        b = tf.get_variable(name='b', shape=(n_classes, ), initializer=tf.zeros_initializer(), trainable=True)
        outputs = tf.reshape(outputs, (s[0] * s[1], s[2]), name='before_proj')
        outputs = tf.nn.xw_plus_b(outputs, W, b)
        outputs = tf.reshape(outputs, (s[0], s[1], n_classes), name='after_proj')
        log_likelihood, transition_params = tf.contrib.crf.crf_log_likelihood(
            inputs=outputs,
            tag_indices=labels,
            sequence_lengths=seq_lens
        )
        loss = tf.reduce_mean(-log_likelihood)
        decode_tags, best_scores = tf.contrib.crf.crf_decode(
            potentials=outputs,
            transition_params=transition_params,
            sequence_length=seq_lens
        )
        mask = tf.sequence_mask(seq_lens, dtype=tf.float32)
        acc = tf.reduce_sum(tf.to_float(tf.equal(decode_tags, labels)) * mask) / tf.reduce_sum(mask)
    return outputs, loss, transition_params, decode_tags, best_scores, acc

In [4]:
tf.reset_default_graph()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
session = tf.Session()
x = tf.placeholder(dtype=tf.float32, shape=(None, None, 1024, 4), name='x')
y = tf.placeholder(dtype=tf.int32, shape=(None, None), name='y')
seq_len = tf.placeholder(dtype=tf.int32, shape=(None,), name='seq_len')
drop_i = tf.placeholder(dtype=tf.float32, shape=(), name='drop_i')
drop_o = tf.placeholder(dtype=tf.float32, shape=(), name='drop_o')
elmo, elmo_l2_reg = elmo_embedding(x)
outputs, loss, transition_params, decode_tags, best_scores, acc = tagger(elmo, y, seq_len, 200, len(pos2idx), drop_i, drop_o)
tag_saver = tf.train.Saver(tf.global_variables())

TypeError: elmo_embedding() missing 1 required positional argument: 'seq_lens'

In [ ]:
def clean_text(x):
    x = str(unicodedata.normalize('NFKC', x.lower()))
    return re.sub('\d+','N', re.sub('[ ]+',' ', re.sub('[\n\r][ \n\r]*',' L ', re.sub(r'(?P<punc>\W)',' \g<punc> ', x))))

def pad_sequence(words):
    maxlen = max(len(x) for x in words)
    arr = np.zeros(shape=(len(words), 1, maxlen))
    for ir in range(len(arr)):
        s = words[ir]
        arr[ir][0][:len(s)] = s
    return arr

def __embed_sequence(sentence):
    unk_char_idx = char2idx['U']
    sentence = [[char2idx.get(x, unk_char_idx) for x in word] for word in sentence]
    seq_len = len(sentence)
    inputs = pad_sequence(sentence)
    embeddings = session.run(lm_model.concated_timewise_output, feed_dict={
        lm_model.inputs: inputs, lm_model.seq_lens: [seq_len], lm_model.reset_state: True
    })
    return embeddings
def embed_sentence(sentence):
#     sentence = clean_text(sentence).split()
    return __embed_sequence(sentence)

In [ ]:
with open('15/checkpoints/model_configs.json', 'r') as inp:
    params = json.load(inp)

lm_model = LanguageModel(**params, is_training=False, is_encoding=True)

lm_model.build_model()
lm_saver = tf.train.Saver([x for x in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, 'LanguageModel') if x not in tf.get_collection('LSTM_SAVED_STATE')])
session.run(tf.global_variables_initializer())
lm_saver.restore(session, '15/checkpoints/test/model.cpkt-315616')
tag_saver.restore(session, './pos_tagger.cpkt-6600')

In [ ]:
def pos_tag(sentence):
    sentence = clean_text(sentence).split()
    em = np.transpose(embed_sentence(sentence), (1, 0, 2, 3))
    tags = session.run(decode_tags, feed_dict={
        x: em, seq_len: [len(em[0])], drop_i: 0.0, drop_o: 0.0
    })
    return [(w, idx2pos[t]) for w, t in zip(sentence, tags[0])]

In [ ]:
pos_tag("""Thứ Bảy ngày 08/09 làm seminar lúc 9h30 nhé. Thái Thanh trình bày về cái chatbot của em. Vẫn chưa thấy em post cái gì mà em nói hôm thứ Bảy lên đây cho tôi xem.

Duc Trung book phòng nhé.""")